# Configure the Training Pipeline

This notebook suppose that you have already downloaded a pre-trained network, so that you have run the notebook `download_pretrained_network.ipynb`. Now, we will modify the configuration file in order to train the pre-trained network `EfficientDet D1 640x640`

In [1]:
# Create a new directory in models/ to store the training configuration of this model
!mkdir training-workspace/models/efficientdet_d1_v1

In [2]:
# Copy the configuration file into this new folder folder
!cp training-workspace/pre-trained-models/efficientdet_d1_coco17_tpu-32/pipeline.config training-workspace/models/efficientdet_d1_v1/

In [3]:
# Display the current configuration file
!cat training-workspace/models/efficientdet_d1_v1/pipeline.config

model {
  ssd {
    num_classes: 90
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "ssd_efficientnet-b1_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 4
        num_filters: 88
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.

So we will modify this configuration file to make it compatible with our project. TODO